In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [ ]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [12]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [17]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.2217591e-03  1.5247524e-03 -2.4622516e-03 ... -2.4434617e-03
   -1.7677713e-03 -2.6711852e-03]
  [-4.9223891e-04  6.1959992e-03 -3.9856178e-03 ... -6.3368061e-04
    1.5522091e-03 -3.4405878e-03]
  [ 2.4908890e-03  1.2330646e-03 -1.9472009e-03 ... -1.5200907e-03
    8.9847138e-03 -2.2194744e-04]
  ...
  [ 1.5126989e-03 -4.5105349e-03 -5.1400624e-03 ... -1.6923998e-03
    5.8895387e-03 -1.7338274e-03]
  [ 6.0948920e-03 -8.4001645e-03  6.1482436e-04 ... -2.4728621e-03
    2.8851451e-03 -2.4738822e-03]
  [ 9.2289252e-03 -5.0037638e-03 -1.2059130e-03 ... -3.7512495e-03
   -5.0428556e-04 -5.2617583e-03]]

 [[-7.3607666e-03 -3.4376066e-03 -6.0184090e-04 ... -5.9132706e-03
    2.2869899e-03  2.8350810e-04]
  [-6.1949957e-03 -4.9471473e-03  4.1575353e-03 ... -2.9805126e-03
   -2.1657008e-03  2.1809326e-03]
  [-5.3906590e-03 -4.9436502e-03 -3.8592210e-03 ...  1.2833287e-03
   -1.8244773e-03  1.0555030e-03]
  ...
  [ 8.4073432e-03  5.0516455e-03 -4.1666911e-03 ...  7.7504455e

In [18]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00322176  0.00152475 -0.00246225 ... -0.00244346 -0.00176777
  -0.00267119]
 [-0.00049224  0.006196   -0.00398562 ... -0.00063368  0.00155221
  -0.00344059]
 [ 0.00249089  0.00123306 -0.0019472  ... -0.00152009  0.00898471
  -0.00022195]
 ...
 [ 0.0015127  -0.00451053 -0.00514006 ... -0.0016924   0.00588954
  -0.00173383]
 [ 0.00609489 -0.00840016  0.00061482 ... -0.00247286  0.00288515
  -0.00247388]
 [ 0.00922893 -0.00500376 -0.00120591 ... -0.00375125 -0.00050429
  -0.00526176]], shape=(100, 65), dtype=float32)


In [19]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 3.2217591e-03  1.5247524e-03 -2.4622516e-03  9.4225950e-04
  2.3759895e-03 -2.2070254e-03  6.7259080e-04 -5.0046965e-03
  5.6972140e-03  6.5043452e-04  2.7244366e-03 -4.7783130e-03
  1.7517485e-03  5.8677178e-03 -4.6826648e-03  2.4422370e-03
 -5.9176385e-03  4.3382072e-03  3.2972149e-04  4.2118779e-03
 -7.0028193e-04  6.2521352e-03  5.1949680e-04 -3.6555901e-04
 -2.0775767e-03  1.5367118e-03 -2.9101335e-03 -6.0551353e-03
 -5.5260811e-04  3.9057343e-03 -2.9482648e-03  3.2129122e-03
 -2.8012828e-03  8.3785122e-03 -2.8072144e-03  1.3822550e-03
 -1.0625415e-03  8.9015253e-04  3.3288621e-03 -3.4367894e-03
  6.9700303e-03  1.4862898e-03 -5.1507456e-03  3.6827126e-04
 -4.0166588e-03 -4.0871054e-03 -2.3769541e-03 -6.2036896e-03
 -3.2875368e-03 -7.3304283e-05 -1.8304721e-03 -4.8862738e-03
 -3.1771227e-03  7.2774375e-03  2.9896195e-03 -4.1382173e-03
 -3.8111485e-03 -5.8880105e-04 -6.8040723e-03  3.3310934e-03
 -1.6976772e-03  4.2591109e-03 -2.4434617e-03 -1.7677713e-03
 -2.671185

In [20]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'BTpSqInm&DN,x:!r$IOr!JCqbB!zWdfsJCXsWC&$ffgl&gX.;PQLuD-ezGBN;,goLBEbebT$qbFXfx;RSaqAR$dpJOJhEzwd&Q:D'

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 952s 6s/step - loss: 2.5931
Epoch 2/50
172/172 [==============================] - 868s 5s/step - loss: 1.8942
Epoch 3/50
172/172 [==============================] - 996s 6s/step - loss: 1.6468
Epoch 4/50
172/172 [==============================] - 922s 5s/step - loss: 1.5112
Epoch 5/50
172/172 [==============================] - 905s 5s/step - loss: 1.4281
Epoch 6/50
172/172 [==============================] - 897s 5s/step - loss: 1.3704
Epoch 7/50
172/172 [==============================] - 952s 6s/step - loss: 1.3247
Epoch 8/50
172/172 [==============================] - 960s 6s/step - loss: 1.2856
Epoch 9/50
172/172 [==============================] - 1027s 6s/step - loss: 1.2493
Epoch 10/50
172/172 [==============================] - 1264s 7s/step - loss: 1.2143
Epoch 11/50
172/172 [==============================] - 1144s 7s/step - loss: 1.1795
Epoch 12/50
172/172 [==============================] - 1006s 6s/step - loss: 1.1443
Epoch 13/

In [25]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [26]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))


# Load model weights from the checkpoint
checkpoint_num = 10
checkpoint_dir = './training_checkpoints'
model.load_weights(os.path.join(checkpoint_dir, "ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [29]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string:  juliet


juliety,
A slaughter-men, for that our fall eyes
To this end retrected:
The flower of the fums ade me as this
your path and the meass that will stay tilk.

BISHOP OF CAMILLO:
I know it age;
And where, sweet son, it stand us deal friends.'
And sometimes custom, knew our man's that kiss
'Thanks they then from my pence whipped with thy scopot: at the
Towers of no less, that I may part of pricket,
whether 'tis as you are heard's a hour of send;
When Isacret and Sut, his eyes son 'Now Bune as he?

CURTIS:
Thou art this look'd your honour's poor words?

CORIOLANUS:
Why should be ot bound? it was,
Stand's powerful witness, that Lend with thy veisure;
The son 'When mane the hundred sides Romeo,
you cannot good and Bolingbroke in her days she is.

QUEEN ELIZABETH:
Sheep hot, or has needly by the gate
Awa
